In [48]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [49]:
# Function to login to LinkedIn
def login_to_linkedin(driver, email, password):
    driver.get('https://www.linkedin.com/login')
    time.sleep(2)
   
    email_input = driver.find_element(By.ID, 'username')
    password_input = driver.find_element(By.ID, 'password')
   
    email_input.send_keys(email)
    password_input.send_keys(password)
   
    login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')
    login_button.click()
    time.sleep(10)

In [50]:
# Function to search for a hashtag and scroll
def search_hashtag(driver, hashtag, max_scroll=5):
    search_url = f'https://www.linkedin.com/feed/hashtag/{hashtag}/'
    driver.get(search_url)
    time.sleep(5)
   
    for _ in range(max_scroll):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

In [51]:
# Function to perform a simple search and scroll
def search_simple(driver, query, max_scroll=5):
    search_url = f'https://www.linkedin.com/search/results/all/?keywords={query}'
    driver.get(search_url)
    time.sleep(5)

    for _ in range(max_scroll):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

In [52]:
# # Function to scrape the first post under the hashtag
# def scrape_first_post(driver):
#     post_text = ""
#     try:
#         first_post = driver.find_element(By.CLASS_NAME, 'occludable-update')
#         post_text = first_post.find_element(By.CLASS_NAME, 'feed-shared-update-v2__description').text
#     except Exception as e:
#         print(f"Error finding or extracting the first post: {e}")
   
#     return post_text

In [53]:
# Function to extract URLs from search results
def extract_urls(driver):
    urls = []
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        containers = soup.find_all('div', {'id': 'fie-impression-container'})
        
        for container in containers:
            link_tag = container.find('a', {'class': 'app-aware-link update-components-actor__meta-link'})
            if link_tag and 'href' in link_tag.attrs:
                urls.append(link_tag['href'])
    except Exception as e:
        print(f"Error extracting URLs: {e}")

    return urls

In [59]:
def save_to_csv(url, post, filename='two_scraped_data.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['url', 'post'])  # Write the header
        writer.writerow([url, post])     # Write the data

In [55]:
# Function to go to the next page of search results and scroll
def go_to_next_page(driver, max_scroll=5):
    try:
        next_button = driver.find_element(By.XPATH, '//button[contains(@class, "artdeco-pagination__button--next")]')
        next_button.click()
        time.sleep(5)  # wait for the next page to load
        for _ in range(max_scroll):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
        return True
    except Exception as e:
        print(f"Error going to the next page: {e}")
        return False

In [56]:
# Updated function to scrape post content
def scrape_post_content(driver):
    post_elements = driver.find_elements(By.XPATH, '//div[@class="update-components-text relative update-components-update-v2__commentary"]/div[@class="break-words tvm-parent-container"]/span[@dir="ltr"]')
    posts = []
    for element in post_elements:
        posts.append(element.text)
    return posts

In [60]:
# Main function
def main():
    email = 'kingofkingsaxox@proton.me'
    password = 'Alok@123'
    query = input("Enter the search query: ")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        login_to_linkedin(driver, email, password)
        search_simple(driver, query)
        
        all_urls = []
        while True:
            urls = extract_urls(driver)
            all_urls.extend(urls)
            posts = scrape_post_content(driver)
            
            if not go_to_next_page(driver):
                break
        
        save_to_csv(all_urls,posts)
        print(f"Data scraped and saved to scraped_data.csv")
    finally:
        driver.quit()

In [61]:
main()

Error going to the next page: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00E6C203+27395]
	(No symbol) [0x00E03E04]
	(No symbol) [0x00D01B7F]
	(No symbol) [0x00CDE483]
	(No symbol) [0x00D6A06F]
	(No symbol) [0x00D7C3D6]
	(No symbol) [0x00D63736]
	(No symbol) [0x00D37541]
	(No symbol) [0x00D380BD]
	GetHandleVerifier [0x01123AB3+2876339]
	GetHandleVerifier [0x01177F7D+3221629]
	GetHandleVerifier [0x00EED674+556916]
	GetHandleVerifier [0x00EF478C+585868]
	(No symbol) [0x00E0CE44]
	(No symbol) [0x00E09858]
	(No symbol) [0x00E099F7]
	(No symbol) [0x00DFBF4E]
	BaseThreadInitThunk [0x763F5D49+25]
	RtlInitializeExceptionChain [0x77DABD2B+107]
	RtlGetAppContainerNamedObjectPath [0x77DABCB1+561]

Data scraped and saved to scraped_data.csv
